In [1]:
import numpy as np
import pandas as pd
# import scipy
# from scipy import optimize
from utils import preprocess, score, features
from utils.features import prefix, suffix

In [2]:
# params, maybe need to remove that
model_name = 'model.pickle'
model_matrices = 'model_matrices.pickle'
model_preprocess = 'model_preprocess.pickle'
load_model = False
load_matrices = False
load_preprocess = False
verbose = 1

# variables
limit_common_words = 5
threshold = 0
args = None
lamda = 0.1

# data files
comp1_path = 'data/comp1.words'
comp2_path = 'data/comp2.words'
test1_path = 'data/test1.wtag'
train1_path = 'data/train1.wtag'
train2_path = 'data/train2.wtag'

In [3]:
# %%time
# train1_statistics = preprocess.feature_statistics_class(train1_path, limit_common_words, lamda)
# train1_statistics.get_statistics()
# train2_statistics = preprocess.feature_statistics_class(train2_path, limit_common_words, lamda)
# train2_statistics.get_statistics()
# id_class = preprocess.feature2id_class(train1_statistics, threshold)
# id_class.get_features()

In [4]:
# all_tags = set(train1_statistics.tags)
# tags_dict = dict(zip(all_tags, range(len(all_tags))))
# display(tags_dict)

## Create FeatureGroup params

In [5]:
feature_groups_dicts = {
    ('t2', 't1', 't'): dict(),
    ('t1', 't'): dict(),
    ('w[i]', 't'): dict(),
    ('w[i-1]', 't'): dict(),
    ('w[i+1]', 't'): dict(),
    ('prefix(w[i], 3)', 't'): dict(),
    ('prefix(w[i], 2)', 't'): dict(),
    ('prefix(w[i+1], 3)', 't'): dict(),
    ('prefix(w[i-1], 3)', 't'): dict(),
    ('suffix(w[i], 3)', 't'): dict(),
    ('suffix(w[i], 2)', 't'): dict(),
    ('suffix(w[i+1], 3)', 't'): dict(),
    ('suffix(w[i-1], 3)', 't'): dict(),

}

single_features_list = [
    'w[i] == w[i].lower()',
    'any(char.isdigit() for char in w[i])',

]

In [6]:
%%time
# get statistics
train1_statistics = preprocess.feature_statistics_class(train1_path, limit_common_words, lamda)
train1_statistics.get_statistics()


for sentence in range(len(train1_statistics.sentences_with_only_word)):
    w = train1_statistics.sentences_with_only_word[sentence]
    tags = train1_statistics.sentences_with_only_tag[sentence]
    t1 = '*'
    t = '*'
    for i in range(len(w)):
        t2, t1, t = t1, t, tags[i]
        for hash_rule in feature_groups_dicts:
            try:
                key = tuple(eval(r) for r in hash_rule)
                if key not in feature_groups_dicts[hash_rule]:
                    feature_groups_dicts[hash_rule][key] = 1
                else:
                    feature_groups_dicts[hash_rule][key] += 1
            except:
                pass

# init FeatureGroups and Features
features_list = []
for hash_rule in feature_groups_dicts:
    features_list.append(features.FeatureGroup(hash_rule, feature_groups_dicts[hash_rule]))
    
for feat in single_features_list:
    features_list.append(features.Feature(feat))
    
# init FeatureVector
feature_vector = features.FeatureVector(features_list)

# display statistics
total_feats = 0
for feat in features_list:
    feat_len = len(feat)
    total_feats += feat_len
    print('feat_group:', feat, '| feats:', feat_len)
print('feat_groups:', len(features_list), '| total_feats:', total_feats)
assert len(feature_vector) == total_feats

feat_group: FeatureGroup(('t2', 't1', 't'), dict(...)) | feats: 8150
feat_group: FeatureGroup(('t1', 't'), dict(...)) | feats: 1060
feat_group: FeatureGroup(('w[i]', 't'), dict(...)) | feats: 15415
feat_group: FeatureGroup(('w[i-1]', 't'), dict(...)) | feats: 32170
feat_group: FeatureGroup(('w[i+1]', 't'), dict(...)) | feats: 30777
feat_group: FeatureGroup(('prefix(w[i], 3)', 't'), dict(...)) | feats: 8061
feat_group: FeatureGroup(('prefix(w[i], 2)', 't'), dict(...)) | feats: 3009
feat_group: FeatureGroup(('prefix(w[i+1], 3)', 't'), dict(...)) | feats: 17800
feat_group: FeatureGroup(('prefix(w[i-1], 3)', 't'), dict(...)) | feats: 18458
feat_group: FeatureGroup(('suffix(w[i], 3)', 't'), dict(...)) | feats: 4500
feat_group: FeatureGroup(('suffix(w[i], 2)', 't'), dict(...)) | feats: 1659
feat_group: FeatureGroup(('suffix(w[i+1], 3)', 't'), dict(...)) | feats: 13488
feat_group: FeatureGroup(('suffix(w[i-1], 3)', 't'), dict(...)) | feats: 13844
feat_group: Feature(('w[i] == w[i].lower()',))

In [15]:
# features = []

# # one-to-one features
# for word in strange_words:
#     features.append(Feature(f'w[i] == "{word}"', t=train1_model.TagsPerWord[word][0]))
#     print(word, train1_model.WordCount[word], train1_model.TagsPerWord[word])

. 4914 ['.']
`` 838 ['``']
, 6044 [',']
'' 813 ["''"]
$ 773 ['$']
-- 256 [':']
-LRB- 148 ['-LRB-']
-RRB- 148 ['-RRB-']
: 149 [':']
-LCB- 30 ['-LRB-']
-RCB- 30 ['-RRB-']
; 133 [':']
... 19 [':']
C$ 13 ['$']
US$ 15 ['$']
A$ 1 ['$']
? 43 ['.']
! 5 ['.']
` 8 ['``']
- 8 [':']
# 7 ['#']
HK$ 6 ['$']


In [7]:
t2, t1, w, i, t = 'NN', 'VB', ['preprocessing' for _ in range(200)], 100, 'NN'
print(feat(t2, t1, w, i, t))

None
